In [1]:
import numpy as np
import os
import tensorflow as tf
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from PIL import Image
import numpy as np

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
labels = []
list_imgs = []
for element_type in os.listdir('./resized_img//'):
    for pic in os.listdir('./resized_img//' + element_type):
        try:
            img = Image.open('./resized_img/' + element_type + '/' + pic)
            img = np.array(img).flatten()
            labels.append(element_type)
            list_imgs.append(img)
        except IsADirectoryError:
            print(".ipynb_checkpoints error")

In [3]:
images = np.array(list_imgs)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train = X_train.astype("float") / 255
X_test = X_test.astype("float") / 255

In [5]:
X_train = X_train.reshape(-1, 128, 128, 3)
X_test = X_test.reshape(-1, 128, 128, 3)

In [6]:
lb = LabelBinarizer()
y_train_OHE = lb.fit_transform(y_train)
y_test_OHE = lb.transform(y_test)

In [7]:
X_train.shape

(1697, 128, 128, 3)

# convnet

In [27]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3)))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
#model.add(tf.keras.layers.Dense(12, activation='softmax'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(12))
model.add(tf.keras.layers.Activation("softmax"))

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 40, 40, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               692352    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
__________

In [29]:
epochs = 30
batch_size=128

In [30]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics=['accuracy'])

model.fit(X_train, y_train_OHE,
          batch_size=batch_size,
          epochs=epochs,
          verbose=True,
          validation_data=(X_test, y_test_OHE))
score = model.evaluate(X_test, y_test_OHE, verbose=True)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1697 samples, validate on 425 samples
Epoch 1/30
1697/1697 [==============================] - 2s 1ms/step - loss: 2.7720 - acc: 0.2634 - val_loss: 1.5283 - val_acc: 0.5247
Epoch 2/30
1697/1697 [==============================] - 1s 831us/step - loss: 1.0881 - acc: 0.6470 - val_loss: 0.6760 - val_acc: 0.7694
Epoch 3/30
1697/1697 [==============================] - 1s 804us/step - loss: 0.5696 - acc: 0.7996 - val_loss: 0.5119 - val_acc: 0.8071
Epoch 4/30
1697/1697 [==============================] - 1s 810us/step - loss: 0.3326 - acc: 0.8786 - val_loss: 0.4074 - val_acc: 0.8635
Epoch 5/30
1697/1697 [==============================] - 1s 809us/step - loss: 0.2437 - acc: 0.9116 - val_loss: 0.3599 - val_acc: 0.8941
Epoch 6/30
1697/1697 [==============================] - 1s 809us/step - loss: 0.1549 - acc: 0.9470 - val_loss: 0.3764 - val_acc: 0.8965
Epoch 7/30
1697/1697 [==============================] - 1s 811us/step - loss: 0.1201 - acc: 0.9558 - val_loss: 0.4249 - val_acc: 0.8753
Epo

# Challenge 6

In [31]:
! curl -O https://challenge.blob.core.windows.net/challengefiles/summit_post_urls_selected.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25750  100 25750    0     0  29030      0 --:--:-- --:--:-- --:--:-- 28997


In [32]:
! head summit_post_urls_selected.txt 